In [1]:
# libs
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import math
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
from tabulate import tabulate
import numpy as np
from scipy.stats import fisher_exact
from tabulate import tabulate

### Load RCT Results
1. Load RCT Result
2. Prepare data for accuracy

In [14]:
INPUT_DIR = os.path.join(os.getcwd(), "rct_results")

df_dict = {}
pre_fix = "10_concepts"
target_questions = ['concept', 'prediction', 'limitation']
for group in ["non_native", "native"]:
    if group not in df_dict.keys():
        df_dict[group] = dict()
    for condition in ['definition', 'story']:
        # file_path = os.path.join(INPUT_DIR, '_'.join([pre_fix, group, condition, 'scores.tsv']))
        # df = pd.read_csv(file_path, sep="\t")
        # df_dict[group][condition] = df
        for study in ["scores", "scores_followup"]:
            file_path = os.path.join(INPUT_DIR, '_'.join([pre_fix, group, condition, study + ".tsv"]))
            study_df = pd.read_csv(file_path, sep='\t')
            df_dict[group][condition][study] = study_df.copy()
            # followup_df_dict[group + "_" + condition][study] = study_df.copy()

for group in df_dict.keys():
    for condition in df_dict[group].keys():
        for study in ["scores", "scores_followup"]:
            raw_df = df_dict[group][condition][study]
            for q in target_questions:
                # definition
                real_answer = np.array(raw_df[q + "_question_answer"].values)
                user_answer = np.array(raw_df[q + "Q"].values)
                correctness = np.multiply((real_answer == user_answer), 1)
                raw_df[q + "_rst"] = correctness
            df_dict[group][condition][study] = raw_df

### Statistical Analysis
1. Chi-Squared for the question accuracy in the test and the follow-up test
2. Mann-Whitney test for the relevance score and interest in law

In [18]:
# Mann Whitney
def mannwhitneyu_test(data1, data2):
    u1, p = mannwhitneyu(data1, data2, method="asymptotic")
    threshold = 0.05
    if p < threshold:
        # Statistically different
        n = True
    else:
        n = False
    return n, u1, p

# Chi-Squared test of accuracy
print("Chi Squared for Accuracy")
for group in ["native", "non_native"]:
    print("For Group ", group)
    for question in [q + "_rst" for q in target_questions]:
        print("*************")
        print("Question: ", question)
        contingency_table = []
        for condition in ["story", "definition"]:  
            data = df_dict[group][condition]
            incorrect = data[question].value_counts()[0]
            correct = data[question].value_counts()[1]
            contingency_table.append([correct, incorrect])
        stat, p, dof, expected = chi2_contingency(contingency_table)
        print(stat, p)

# Mann-Whitney
print("\n")
print("Mann-Whitney U Test")
for group in ["native", "non_native"]:
    print("For Group ", group)
    for metric in ["relevance", "law_interest"]:  
        data1 = df_dict[group]['story'][metric]
        data2 = df_dict[group]['definition'][metric]
        print("Metric: ", metric)
        print("Story Condition MD(STD): ", np.mean(data1), np.std(data1))
        print("Definition Condition MD(STD): ", np.mean(data2), np.std(data2))
        n, u, p = mannwhitneyu_test(data1, data2)
        print(u, p)
        print("\n")

Chi Squared for Accuracy
For Group  native
*************
Question:  concept_rst
0.48339200428194906 0.4868898821253125
*************
Question:  prediction_rst
0.6537854593213362 0.41876252154968585
*************
Question:  limitation_rst
2.0120481800016 0.1560545129081714
For Group  non_native
*************
Question:  concept_rst
0.20182537119577626 0.6532514758125724
*************
Question:  prediction_rst
4.2850363890097 0.0384492595157595
*************
Question:  limitation_rst
11.772340686798307 0.0006011733562701144


Mann-Whitney U Test
For Group  native
Metric:  relevance
Story Condition MD(STD):  3.20625 1.3233332677371943
Definition Condition MD(STD):  2.6303030303030304 1.3037207473076875
16421.0 0.00010137547692223972


Metric:  law_interest
Story Condition MD(STD):  3.78125 0.9916644782889019
Definition Condition MD(STD):  3.6666666666666665 1.119162746219357
13800.0 0.45934243459962987


For Group  non_native
Metric:  relevance
Story Condition MD(STD):  3.1882352941176473 

### Followup Assessment result

In [24]:
DATA_DIR = os.path.join(os.getcwd(), "rct_results")

total_participant = []
followup_df_dict = {}
pre_fix = "10_concepts"
for group in ["non_native", "native"]:
    for condition in ['definition', 'story']:
        if group + "_" + condition not in followup_df_dict.keys():
            followup_df_dict[group + "_" + condition] = dict()
        for study in ["scores", "scores_followup"]:
            file_path = os.path.join(DATA_DIR, '_'.join([pre_fix, group, condition, study + ".tsv"]))
            study_df = pd.read_csv(file_path, sep='\t')
            followup_df_dict[group + "_" + condition][study] = study_df.copy()

In [29]:
total_respondant = 0
total_participant = 0
focused_population_result = dict()
for _type in followup_df_dict.keys():
    # old_file = os.path.join(os.getcwd(), "rct_exp", '_'.join([pre_fix, _type, 'scores.csv']))
    # old_df = pd.read_csv(old_file)
    print(_type)
    # print("Reply Rate: ", round(len(followup)/len(old_df), 2))

    original = followup_df_dict[_type]['scores']
    followup = followup_df_dict[_type]['scores_followup']
    merged_df = pd.merge(original, followup, on=['PROLIFIC_PID', 'q_id'])
    print(merged_df.columns)
    # for question in ['concept', 'q2_pred', 'q3_pred']:
    for question in ['concept_question_answer', 'prediction_question_answer', 'limitation_question_answer']:
        print("Question: ", question)
        retention_population = merged_df[merged_df[question + "_x"] == True]
        print(round(len(retention_population[retention_population[question + "_y"] == True])/len(retention_population), 4)*100)

        # save the data:
        if question not in focused_population_result.keys():
            focused_population_result[question] = dict()
        retention_population[question + "_y"] = retention_population[question + "_y"].replace({True: 1, False: 0})
        focused_population_result[question][_type] = list(retention_population[question + "_y"].values)
        

    total_respondant += len(followup)
    total_participant += len(original)
    print("\n")
print("Total Responding Rate: ", round(total_respondant/total_participant, 2))

non_native_definition
Index(['Unnamed: 0_x', 'PROLIFIC_PID', 'q_id', 'law_background', 'age',
       'gender', 'edu_background', 'law_interest', 'relevance', 'conceptQ_x',
       'predictionQ_x', 'limitationQ_x', 'familiarity_concept_x',
       'perceived_difficulty', 'batch_id', 'concept_x',
       'concept_question_answer_x', 'prediction_question_answer_x',
       'limitation_question_answer_x', 'Unnamed: 0_y', 'conceptQ_y',
       'predictionQ_y', 'limitationQ_y', 'familiarity_concept_y', 'concept_y',
       'concept_question_answer_y', 'prediction_question_answer_y',
       'limitation_question_answer_y'],
      dtype='object')
Question:  concept_question_answer
0.0
Question:  prediction_question_answer
0.0
Question:  limitation_question_answer
0.0


non_native_story
Index(['Unnamed: 0_x', 'PROLIFIC_PID', 'q_id', 'law_background', 'age',
       'gender', 'edu_background', 'law_interest', 'relevance', 'conceptQ_x',
       'predictionQ_x', 'limitationQ_x', 'familiarity_concept_x',
  

/var/folders/zl/b_kmp9gj7nx2fx9721_dt5mc0000gn/T/ipykernel_13318/2995976724.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_population[question + "_y"] = retention_population[question + "_y"].replace({True: 1, False: 0})
/var/folders/zl/b_kmp9gj7nx2fx9721_dt5mc0000gn/T/ipykernel_13318/2995976724.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_population[question + "_y"] = retention_population[question + "_y"].replace({True: 1, False: 0})
/var/folders/zl/b_kmp9gj7nx2fx9721_dt5mc

In [22]:
comparison = [
    ["non_native_definition", "non_native_story"],
    ["native_definition", "native_story"]
]
for question in ['q1_pred', 'q2_pred', 'q3_pred']:
    print(question)
    for case1, case2 in comparison:
        print(case1, case2)
        list1 = focused_population_result[question][case1]
        list2 = focused_population_result[question][case2]
        table = []
        total = 0
        for _list in [list1, list2]:  
            incorrect = _list.count(0)
            correct = _list.count(1)
            table.append([correct, incorrect])
            print(correct/(correct+incorrect))
            total += (correct + incorrect)
        # print(tabulate(table))
        print(total)
        print("Chi Squared")
        stat, p, dof, expected = chi2_contingency(table)
        print(stat, p)
    print("\n")

q1_pred
non_native_definition non_native_story


KeyError: 'q1_pred'